# Exercício 4

Obtenha as séries de cotações de venda (*ask*) para as moedas USD, EUR, CNY, GBP e CHF, cotadas em BRL, para o período de 2010-12-31 a 2020-01-01 (10 anos de dados). Estas séries possuem valores faltantes, é necessário fazer o tratamento. Para cada uma destas séries faça:
1. Limpeza dos dados faltantes
1. Cálculo do log-retorno diário
1. Dividir as séries em 5 períodos de 2 anos cada e calcule as estatísticas: retorno esperado e volatilidade anualizados, para cada sub-período.
1. O que você observa? As medidas permanecem *constantes* ao longo do tempo?

In [53]:
import pandas as pd
import numpy as np
from bcb import currency

In [54]:
symbols = ['USD', 'EUR', 'CNY', 'GBP', 'CHF']

In [71]:
data = currency.get(symbols, start_date='2010-12-31', end_date='2020-01-01')

In [72]:
data.head()

USD             EUR             CNY             GBP          \
               bid     ask     bid     ask     bid     ask     bid     ask   
Date                                                                         
2010-12-31  1.6654  1.6662  2.2266  2.2280  0.2521  0.2522  2.5859  2.5876   
2011-01-03  1.6502  1.6510  2.2086  2.2099  0.2498  0.2499  2.5565  2.5581   
2011-01-04  1.6548  1.6556  2.2040  2.2054  0.2503  0.2505  2.5803  2.5819   
2011-01-05  1.6705  1.6713  2.2000  2.2014  0.2523  0.2525  2.5883  2.5898   
2011-01-06  1.6849  1.6857  2.1914  2.1926  0.2543  0.2544  2.6054  2.6069   

               CHF          
               bid     ask  
Date                        
2010-12-31  1.7814  1.7828  
2011-01-03  1.7683  1.7698  
2011-01-04  1.7459  1.7470  
2011-01-05  1.7304  1.7316  
2011-01-06  1.7451  1.7463

## Limpeza dos dados faltantes

In [73]:
p = data.loc[:, pd.IndexSlice[:, 'ask']]
p.columns = symbols

In [74]:
p.shape

(2262, 5)

In [75]:
p.dropna().shape

(2262, 5)

In [76]:
p.head()

,USD,EUR,CNY,GBP,CHF
Date,,,,,
2010-12-31,1.6662,2.2280,0.2522,2.5876,1.7828
2011-01-03,1.6510,2.2099,0.2499,2.5581,1.7698
2011-01-04,1.6556,2.2054,0.2505,2.5819,1.7470
2011-01-05,1.6713,2.2014,0.2525,2.5898,1.7316
2011-01-06,1.6857,2.1926,0.2544,2.6069,1.7463


## Cálculo do log-retorno diário

In [77]:
r = np.log(p).diff()

## Estatísticas em subperíodos de 2 anos

Dividir as séries em 5 períodos de 2 anos cada e calcule as estatísticas: retorno esperado e volatilidade anualizados, para cada sub-período.

### Criando os subperíodos

In [78]:
r['group'] = np.ceil(r.index.year / 2).astype('int32') - 1005

### Calculando o retorno esperado anualizado para cada subperíodo

In [79]:
r.dropna()\
    .groupby('group')\
    .apply(lambda x: x.mean() * 252)

,USD,EUR,CNY,GBP,CHF,group
group,,,,,,
1,0.102466,0.095601,0.131919,0.122552,0.112894,252.0
2,0.130598,0.089647,0.132645,0.112531,0.091677,504.0
3,0.102890,0.031917,0.046432,-0.012808,0.089402,756.0
4,0.087388,0.128991,0.092253,0.104232,0.103995,1008.0
5,0.039290,0.020323,0.026851,0.070759,0.058726,1260.0


### Calculando o risco esperado anualizado para cada subperíodo

In [80]:
r.dropna()\
    .groupby('group')\
    .apply(lambda x: x.std() * np.sqrt(252))

,USD,EUR,CNY,GBP,CHF,group
group,,,,,,
1,0.121111,0.110823,0.120044,0.107925,0.146576,0.0
2,0.116278,0.126206,0.116281,0.124278,0.134139,0.0
3,0.181305,0.189003,0.176492,0.189607,0.215138,0.0
4,0.132038,0.135758,0.129635,0.146603,0.139689,0.0
5,0.110670,0.108853,0.102331,0.128918,0.117721,0.0
